In [1]:
# for google cloud gpu only
!pip install keras

In [2]:
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
# CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

In [0]:
#constant
BATCH_SIZE = 128
NB_EPOCH = 25   # CPU 1, GPU 25
NB_CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = RMSprop()

In [5]:
#load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


#### Now let's do a one-hot encoding and normalize the images:

In [0]:
# convert to categorical
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)

In [0]:
# float and normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

### Let build the network for CIFAR-10

In [8]:
# network for CIFAR-10
model = Sequential()

# Keras Con2D format
# keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', data_format=None, dilation_rate=(1, 1), activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)

# CONV (32 filters of size [3,3]) => RELU
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))

# max pooling with 0.25 dropout
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# CONV (32 filters of size 3)
model.add(Conv2D(32, kernel_size=3, padding='same'))
model.add(Activation('relu'))

# max pooling with 0.25 dropout
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
# CONV (64 filters of size 3)  
model.add(Conv2D(64, kernel_size=3, padding='same'))
model.add(Activation('relu'))

# CONV (64 filters of size 3, strides 3)
model.add(Conv2D(64, 3, 3))
model.add(Activation('relu'))

# max pooling with 0.25 dropout
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Dense layer of 512 => RELU
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))

# Dense layer of NB_CLASSES=10 => softmax
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`


In [9]:
# print model summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 16, 16, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 32)          0         
__________

### Let's train the model

In [10]:
# train it
model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH, validation_split=VALIDATION_SPLIT, verbose=VERBOSE)

Train on 40000 samples, validate on 10000 samples
Epoch 1/25
40000/40000 [==============================] - 8s 207us/step - loss: 1.9161 - acc: 0.2933 - val_loss: 1.7209 - val_acc: 0.3733
Epoch 2/25
40000/40000 [==============================] - 7s 172us/step - loss: 1.5677 - acc: 0.4274 - val_loss: 1.3531 - val_acc: 0.5139
Epoch 3/25
40000/40000 [==============================] - 7s 170us/step - loss: 1.4100 - acc: 0.4916 - val_loss: 1.2847 - val_acc: 0.5296
Epoch 4/25
 6912/40000 [====>.........................] - ETA: 5s - loss: 1.3273 - acc: 0.5168

40000/40000 [==============================] - 7s 171us/step - loss: 1.3050 - acc: 0.5309 - val_loss: 1.2012 - val_acc: 0.5733
Epoch 5/25
40000/40000 [==============================] - 7s 169us/step - loss: 1.2219 - acc: 0.5645 - val_loss: 1.0846 - val_acc: 0.6155
Epoch 6/25
40000/40000 [==============================] - 7s 170us/step - loss: 1.1515 - acc: 0.5901 - val_loss: 1.1023 - val_acc: 0.6198
Epoch 7/25
23936/40000 [================>.............] - ETA: 2s - loss: 1.0999 - acc: 0.6128

40000/40000 [==============================] - 7s 168us/step - loss: 1.0993 - acc: 0.6120 - val_loss: 0.9564 - val_acc: 0.6645
Epoch 8/25
40000/40000 [==============================] - 7s 169us/step - loss: 1.0562 - acc: 0.6281 - val_loss: 0.9646 - val_acc: 0.6623
Epoch 9/25
40000/40000 [==============================] - 7s 168us/step - loss: 1.0121 - acc: 0.6454 - val_loss: 0.9049 - val_acc: 0.6856
Epoch 10/25
27008/40000 [===================>..........] - ETA: 2s - loss: 0.9837 - acc: 0.6528

40000/40000 [==============================] - 7s 169us/step - loss: 0.9834 - acc: 0.6539 - val_loss: 0.9244 - val_acc: 0.6875
Epoch 11/25
40000/40000 [==============================] - 7s 169us/step - loss: 0.9584 - acc: 0.6632 - val_loss: 0.8849 - val_acc: 0.6957
Epoch 12/25
40000/40000 [==============================] - 7s 169us/step - loss: 0.9358 - acc: 0.6738 - val_loss: 0.8270 - val_acc: 0.7111
Epoch 13/25
27008/40000 [===================>..........] - ETA: 2s - loss: 0.9121 - acc: 0.6794

40000/40000 [==============================] - 7s 169us/step - loss: 0.9114 - acc: 0.6799 - val_loss: 0.8532 - val_acc: 0.7037
Epoch 14/25
40000/40000 [==============================] - 7s 169us/step - loss: 0.8897 - acc: 0.6862 - val_loss: 0.9549 - val_acc: 0.6817
Epoch 15/25
40000/40000 [==============================] - 7s 169us/step - loss: 0.8753 - acc: 0.6955 - val_loss: 0.8595 - val_acc: 0.7083
Epoch 16/25
26624/40000 [==================>...........] - ETA: 2s - loss: 0.8571 - acc: 0.6991

40000/40000 [==============================] - 7s 170us/step - loss: 0.8582 - acc: 0.6993 - val_loss: 0.8145 - val_acc: 0.7185
Epoch 17/25
40000/40000 [==============================] - 7s 170us/step - loss: 0.8460 - acc: 0.7048 - val_loss: 0.8302 - val_acc: 0.7127
Epoch 18/25
40000/40000 [==============================] - 7s 171us/step - loss: 0.8346 - acc: 0.7078 - val_loss: 0.8701 - val_acc: 0.7007
Epoch 19/25
26624/40000 [==================>...........] - ETA: 2s - loss: 0.8259 - acc: 0.7127

40000/40000 [==============================] - 7s 170us/step - loss: 0.8336 - acc: 0.7096 - val_loss: 0.7656 - val_acc: 0.7351
Epoch 20/25
40000/40000 [==============================] - 7s 170us/step - loss: 0.8155 - acc: 0.7145 - val_loss: 0.8511 - val_acc: 0.7126
Epoch 21/25
40000/40000 [==============================] - 7s 169us/step - loss: 0.8086 - acc: 0.7210 - val_loss: 0.8001 - val_acc: 0.7281
Epoch 22/25
27008/40000 [===================>..........] - ETA: 2s - loss: 0.7937 - acc: 0.7226

40000/40000 [==============================] - 7s 169us/step - loss: 0.7978 - acc: 0.7211 - val_loss: 0.7982 - val_acc: 0.7308
Epoch 23/25
40000/40000 [==============================] - 7s 170us/step - loss: 0.8001 - acc: 0.7219 - val_loss: 0.8295 - val_acc: 0.7213
Epoch 24/25
40000/40000 [==============================] - 7s 170us/step - loss: 0.7875 - acc: 0.7254 - val_loss: 0.7894 - val_acc: 0.7284
Epoch 25/25
27008/40000 [===================>..........] - ETA: 2s - loss: 0.7772 - acc: 0.7307

40000/40000 [==============================] - 7s 168us/step - loss: 0.7843 - acc: 0.7275 - val_loss: 0.7417 - val_acc: 0.7441


### Evaluate it

In [11]:
score = model.evaluate(X_test, Y_test, batch_size=BATCH_SIZE, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 1s 54us/step
Test score: 0.7556726346015931
Test accuracy: 0.7385


### **Let's save model**

In [0]:
# save model on disk
model.save('keras_CIFAR10.h5')

### We need to increase training data.. 
### Data Augmentation!!

In [0]:
# imports
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [0]:
# constants
NUM_TO_AUGMENT=5

In [15]:
# augumenting
print("Augmenting training set images...")
datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, 
                             height_shift_range=0.2, zoom_range=0.2, 
                             horizontal_flip=True,fill_mode='nearest')

Augmenting training set images...


In [0]:
datagen.fit(X_train)

### Let's retrain the model

In [17]:
# train
history = model.fit_generator(datagen.flow(X_train, Y_train, 
                                           batch_size=BATCH_SIZE), 
                              samples_per_epoch=X_train.shape[0], 
                              epochs=NB_EPOCH, verbose=VERBOSE)

score = model.evaluate(X_test, Y_test, batch_size=BATCH_SIZE, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

Epoch 1/25
  2/390 [..............................] - ETA: 27s - loss: 1.7866 - acc: 0.4102

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=25, verbose=1, steps_per_epoch=390)`
  after removing the cwd from sys.path.


390/390 [==============================] - 26s 67ms/step - loss: 1.4320 - acc: 0.4967
Epoch 2/25
328/390 [========================>.....] - ETA: 4s - loss: 1.3841 - acc: 0.5215

390/390 [==============================] - 26s 67ms/step - loss: 1.3823 - acc: 0.5213
Epoch 3/25
390/390 [==============================] - 26s 67ms/step - loss: 1.3575 - acc: 0.5272
Epoch 4/25
 68/390 [====>.........................] - ETA: 21s - loss: 1.3442 - acc: 0.5378

390/390 [==============================] - 26s 67ms/step - loss: 1.3490 - acc: 0.5329
Epoch 5/25
367/390 [===========================>..] - ETA: 1s - loss: 1.3362 - acc: 0.5369

...

390/390 [==============================] - 26s 66ms/step - loss: 1.3133 - acc: 0.5447
Epoch 19/25
363/390 [==========================>...] - ETA: 1s - loss: 1.3173 - acc: 0.5440

390/390 [==============================] - 26s 66ms/step - loss: 1.3174 - acc: 0.5438
Epoch 20/25
390/390 [==============================] - 26s 66ms/step - loss: 1.3149 - acc: 0.5458
Epoch 21/25
 85/390 [=====>........................] - ETA: 19s - loss: 1.3031 - acc: 0.5543

390/390 [==============================] - 26s 66ms/step - loss: 1.3123 - acc: 0.5470
Epoch 22/25
390/390 [==============================] - 26s 66ms/step - loss: 1.3221 - acc: 0.5430
Epoch 23/25
  2/390 [..............................] - ETA: 24s - loss: 1.3021 - acc: 0.5781

390/390 [==============================] - 26s 66ms/step - loss: 1.3204 - acc: 0.5446
Epoch 24/25
358/390 [==========================>...] - ETA: 2s - loss: 1.3270 - acc: 0.5446

390/390 [==============================] - 26s 65ms/step - loss: 1.3283 - acc: 0.5436
Epoch 25/25
10000/10000 [==============================] - 1s 69us/step
Test score: 1.1982812564849854
Test accuracy: 0.5995


In [19]:
score = model.evaluate(X_test, Y_test, batch_size=BATCH_SIZE, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 1s 61us/step
Test score: 1.1982812564849854
Test accuracy: 0.5995


In [20]:
# list all data in history
print(history.history.keys())

dict_keys(['loss', 'acc'])
